In [1]:
!pip install torchtext pyarrow transformers structlog

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 93.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 191.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 123.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 126.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 12.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 8.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3

In [9]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
tqdm.pandas()

import torch
torch.manual_seed(42)
import torch.nn as nn
import torchtext
import gc
import structlog


import os
import gc
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

from sklearn.preprocessing import LabelEncoder

from keras.models import Model
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, Add, MaxPooling1D, BatchNormalization
from keras.layers import Embedding, Bidirectional, LSTM, CuDNNLSTM, GlobalMaxPooling1D

import tensorflow as tf
from huggingface_hub import hf_hub_url, cached_download

GPU = True
device = torch.device("cuda" if GPU else "cpu")
data_path = '/home/ubuntu/'
logger = structlog.getLogger()
logger.info(f"Getting started with {GPU=} {device=} {data_path=}")

# Load protbert model
from transformers import BertForMaskedLM, BertTokenizer, pipeline
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
protbert_model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
if GPU:
    protbert_model.cuda()

!tree /home/ubuntu/

2023-01-29 23:07:23 [info     ] Getting started with GPU=True device=device(type='cuda') data_path='/home/ubuntu/'
/home/ubuntu/
├── Ensemble_Protein_1000.ipynb
├── parquet_data.zip
├── test_df.parquet
├── tf_model_protbertembs_1_000_class.state_dict
├── train_df.parquet
└── val_df.parquet

0 directories, 6 files


In [3]:
train_df = pd.read_parquet(data_path + 'train_df.parquet')
test_df = pd.read_parquet(data_path + 'test_df.parquet')
val_df = pd.read_parquet(data_path + 'val_df.parquet')
train_df.shape, test_df.shape, val_df.shape

((1086741, 5), (126171, 5), (126171, 5))

In [5]:
NUM_CLASSES = 1_000
top_families = train_df['family_id'].value_counts()[:NUM_CLASSES]
# Convert to numbers
fam2id = {fam: i for i, fam in enumerate(top_families.index)}

def add_and_filter_family_id(df):
    df['family_code'] = df['family_id'].apply(lambda x: fam2id.get(x, np.nan))
    logger.info(f'Removing {df["family_code"].isna().sum():,}/{len(df):,} = {df["family_code"].isna().mean()*100:,.6f}% of rows due to nan famid num.')
    return df.dropna(subset='family_code').reset_index(drop=True)

train_df = add_and_filter_family_id(train_df)
test_df = add_and_filter_family_id(test_df)
val_df = add_and_filter_family_id(val_df)
import gc; gc.collect()
train_df.shape, test_df.shape, val_df.shape

2023-01-29 23:01:13 [info     ] Removing 647,248/1,086,741 = 59.558625% of rows due to nan famid num.
2023-01-29 23:01:13 [info     ] Removing 71,793/126,171 = 56.901348% of rows due to nan famid num.
2023-01-29 23:01:13 [info     ] Removing 71,793/126,171 = 56.901348% of rows due to nan famid num.


((439493, 6), (54378, 6), (54378, 6))

In [34]:
def load_transformer_model():
    # Transformer models from tutorial https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/
    class PositionalEncoding(nn.Module):
        """
        https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        """

        def __init__(self, d_model, vocab_size=5000, dropout=0.1):
            super().__init__()
            self.dropout = nn.Dropout(p=dropout)

            pe = torch.zeros(vocab_size, d_model)
            position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
            div_term = torch.exp(
                torch.arange(0, d_model, 2).float()
                * (-math.log(10000.0) / d_model)
            )
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)
            pe = pe.unsqueeze(0)
            self.register_buffer("pe", pe)

        def forward(self, x):
            x = x + self.pe[:, : x.size(1), :]
            return self.dropout(x)


    class Net(nn.Module):
        """
        Text classifier based on a pytorch TransformerEncoder.
        """

        def __init__(
            self,
            embeddings,
            vocab_size=30,
            embedding_size=1024,
            nhead=8,
            dim_feedforward=2048,
            num_layers=6,
            num_labels=2,
            dropout=0.1,
            activation="relu",
            classifier_dropout=0.1,
        ):

            super().__init__()

            d_model = embedding_size
            assert d_model % nhead == 0, "nheads must divide evenly into d_model"

            self.emb = embeddings

            self.pos_encoder = PositionalEncoding(
                d_model=d_model,
                dropout=dropout,
                vocab_size=vocab_size,
            )

            encoder_layer = nn.TransformerEncoderLayer(
                d_model=d_model,
                nhead=nhead,
                dim_feedforward=dim_feedforward,
                dropout=dropout,
            )
            self.transformer_encoder = nn.TransformerEncoder(
                encoder_layer,
                num_layers=num_layers,
            )
            self.classifier = nn.Linear(d_model, num_labels)
            self.d_model = d_model
            self._agg_type = 1

        def forward(self, x):
            with torch.no_grad():
                embeds = self.emb(x)
                # x = embeds[:,-10:,:] # Only need last ten
                x = embeds
            # x = self.emb(x) * math.sqrt(self.d_model)
            # x = self.pos_encoder(x)
            x = self.transformer_encoder(x)
            if self._agg_type == 0:
                x = x[:, -1, :]
            else:
                x = x.mean(1)
            x = self.classifier(x)
            return x
    tf_model = Net(
        protbert_model.bert.embeddings,
        vocab_size=tokenizer.vocab_size,
        nhead=8,  # the number of heads in the multiheadattention models
        dim_feedforward=50,  # the dimension of the feedforward network model in nn.TransformerEncoder
        num_layers=6,
        num_labels=NUM_CLASSES,
        dropout=0.2,
        classifier_dropout=0.2,
    ).to(device)
    tf_model.load_state_dict(
        torch.load(data_path + 'tf_model_protbertembs_1_000_class.state_dict')
    )
    return tf_model

tf_model = load_transformer_model()
tf_model

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: CUDA out of memory. Tried to allocate 158.00 MiB (GPU 0; 22.06 GiB total capacity; 3.41 GiB already allocated; 16.44 MiB free; 3.42 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [29]:
def load_xgboost():
    # TODO: Load xgboost
    !pip install xgboost
    import xgboost
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    import sklearn.cluster
    # xgboost_gpu_kwargs = {"gpu_id": 0, "tree_method": 'gpu_hist'} if GPU else {}
    model = xgboost.XGBClassifier()  #verbosity=1, **xgboost_gpu_kwargs, **model_kwargs)
    weights = cached_download(hf_hub_url("jonathang/Protein_Family_Models", 'xgboost_1100_mean_embed.model'))
    model.load_model(weights)
    return model

xgboost_model = load_xgboost()
xgboost_model

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 23.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:629: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


[23:30:52] WARNING: ../src/learner.cc:1098: Loading model from XGBoost < 1.0.0, consider saving it again for improved compatibility


/home/ubuntu/.local/lib/python3.8/site-packages/xgboost/sklearn.py:782: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn("Loading a native XGBoost model with Scikit-Learn interface.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', predictor=None, ...)

In [11]:
def load_cnn():
    def residual_block(data, filters, d_rate):
        """
        _data: input
        _filters: convolution filters
        _d_rate: dilation rate
        """

        shortcut = data

        bn1 = BatchNormalization()(data)
        act1 = Activation('relu')(bn1)
        conv1 = Conv1D(filters, 1, dilation_rate=d_rate, padding='same', kernel_regularizer=l2(0.001))(act1)

        #bottleneck convolution
        bn2 = BatchNormalization()(conv1)
        act2 = Activation('relu')(bn2)
        conv2 = Conv1D(filters, 3, padding='same', kernel_regularizer=l2(0.001))(act2)

        #skip connection
        x = Add()([conv2, shortcut])

        return x

    # model
    x_input = Input(shape=(100, 21))
    
    #initial conv
    conv = Conv1D(128, 1, padding='same')(x_input) 
    
    # per-residue representation
    res1 = residual_block(conv, 128, 2)
    res2 = residual_block(res1, 128, 3)
    
    x = MaxPooling1D(3)(res2)
    x = Dropout(0.5)(x)
    
    # softmax classifier
    x = Flatten()(x)
    x_output = Dense(1000, activation='softmax', kernel_regularizer=l2(0.0001))(x)
    
    model2 = Model(inputs=x_input, outputs=x_output)
    model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    weights = cached_download(hf_hub_url("jonathang/Protein_Family_Models", 'model2.h5'))
    model2.load_weights(weights)

    return model2

cnn_model = load_cnn()
cnn_model

/home/ubuntu/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:629: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [15]:
def get_lstm():
    x_input = Input(shape=(100,))
    max_length = 100
    emb = Embedding(21, 128, input_length=max_length)(x_input)
    bi_rnn = CuDNNLSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))(emb)
    x = Dropout(0.3)(bi_rnn)

    # softmax classifier
    x_output = Dense(1000, activation='softmax')(x)

    model1 = Model(inputs=x_input, outputs=x_output)
    model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    weights = cached_download(hf_hub_url("jonathang/Protein_Family_Models", 'model1.h5'))
    model1.load_weights(weights)
    return model1

lstm_model = get_lstm()
lstm_model

In [17]:
# TODO: Write prep funcs per input sequence
class BaseModelIO:
    def prepare_sequence(self, seq):
        raise NotImplemented
    def convert_output(self, out):
        raise NotImplemented

In [24]:
class TFModelIO(BaseModelIO):
    def __init__(self):
        import re
        from transformers import BertForMaskedLM, BertTokenizer
        self._x_aminos = re.compile("[UZOB]")
        self.max_len = 100
        self._tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
    def prepare_sequence(self, seq):
        seq = seq[:self.max_len-10]
        seq = self._x_aminos.sub("X", ' '.join(seq))
        seq = self._tokenizer.encode(seq, add_special_tokens=True, padding='max_length', max_length=self.max_len)
        seq = torch.tensor(seq, device='cuda' if GPU else 'cpu')
        return seq

tf_model_io = TFModelIO()
tf_model_io.prepare_sequence('RRRWWW')

tensor([ 2, 13, 13, 13, 24, 24, 24,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0], device='cuda:0')

In [33]:
class XGBoostModelIO(BaseModelIO):
    def __init__(self):
        from transformers import BertForMaskedLM, BertTokenizer, pipeline
        self._tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
        self._model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert").bert.embeddings
        import re
        self._x_aminos = re.compile("[UZOB]")
    def prepare_sequence(self, seq):
        seq = ' '.join(seq)
        seq = self._x_aminos.sub("X", seq)
        # tokenizer-> token_id
        input_ids = self._tokenizer.encode(seq, add_special_tokens=True)
        # input_ids: [101, 2182, 2003, 2070, 3793, 2000, 4372, 16044, 102]
        input_ids = torch.tensor([input_ids], device='cuda' if GPU else 'cpu')
        with torch.no_grad():
            return self._model(input_ids)

xgboost_model_io = XGBoostModelIO()
xgboost_model_io.prepare_sequence('RRRWWW')

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper__index_select)

In [25]:
class CNNModelIO(BaseModelIO):
    codes = {c: i+1 for i, c in enumerate(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])}

    def integer_encoding(self, data):
        """
        - Encodes code sequence to integer values.
        - 20 common amino acids are taken into consideration
        and remaining four are categorized as 0.
        """
        return np.array([self.codes.get(code, 0) for code in data])

    def prepare_sequence(self, sequence):
        sequence = sequence.strip().upper()
        ie = self.integer_encoding(sequence)
        max_length = 100
        padded_ie = pad_sequences([ie], maxlen=max_length, padding='post', truncating='post')
        all_ohe = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] + [0]*(100-21))
        return to_categorical(np.array([padded_ie[0], all_ohe]))[:1]

cnn_model_io = CNNModelIO()
cnn_model_io.prepare_sequence('RRRWWW')

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtype=float32)